In [1]:
from shapely.wkt import loads
import geojson
import geopandas as gpd
import pandas as pd
import json
from shapely.geometry import Polygon
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go


In [2]:
cropland_fires = pd.read_csv('cropland_fires.csv')

In [3]:
def wkt_to_geojson(wkt):
    # Load the WKT string into a Shapely geometry object
    geometry = loads(wkt)
    
    # Convert the Shapely geometry to GeoJSON format
    geojson_feature = geojson.Feature(geometry=geometry)
    
    # Return the GeoJSON representation
    return geojson.dumps(geojson_feature)

In [4]:
cropland_fires['geojson'] = cropland_fires['st_astext'].apply(wkt_to_geojson)

In [5]:
india_df =  cropland_fires[cropland_fires['country'] == 'India']

In [6]:
india_df.drop(columns=['iso3_country', 'country', 'asset_name','Unnamed: 0', 'gas_ch4','gas_co2','gas_n2o','gas_co2e_20yr','gas_co2e_100yr','gas_n2o'],inplace=True)
india_df

/tmp/ipykernel_16874/1585570770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df.drop(columns=['iso3_country', 'country', 'asset_name','Unnamed: 0', 'gas_ch4','gas_co2','gas_n2o','gas_co2e_20yr','gas_co2e_100yr','gas_n2o'],inplace=True)


,asset_id,start_time,end_time,gas,emissions_quantity,emissions_factor,capacity,activity,st_astext,state,month,year,geojson
1746,1834587053,2015-04-01,2015-04-30 00:00:00,co2,124.098134,1440.0,7.29,1.0,"POLYGON((85.875 21.625,85.875 21.875,86.125 21...",Odisha,4,2015,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1747,1834587053,2015-04-01,2015-04-30 00:00:00,ch4,0.633308,5.7,7.29,1.0,"POLYGON((85.875 21.625,85.875 21.875,86.125 21...",Odisha,4,2015,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1748,1834587053,2015-04-01,2015-04-30 00:00:00,n2o,0.007539,0.1,7.29,1.0,"POLYGON((85.875 21.625,85.875 21.875,86.125 21...",Odisha,4,2015,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1749,1834587053,2015-04-01,2015-04-30 00:00:00,co2e_100yr,143.825666,0.1,7.29,1.0,"POLYGON((85.875 21.625,85.875 21.875,86.125 21...",Odisha,4,2015,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1750,1834587053,2015-04-01,2015-04-30 00:00:00,co2e_20yr,177.580962,0.1,7.29,1.0,"POLYGON((85.875 21.625,85.875 21.875,86.125 21...",Odisha,4,2015,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998739,1834585365,2021-07-01,2021-07-31 00:00:00,co2,8.341301,1440.0,0.49,0.5,"POLYGON((76.125 28.125,76.125 28.375,76.375 28...",Haryana,7,2021,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
9998743,1834585365,2021-07-01,2021-07-31 00:00:00,ch4,0.042568,5.7,0.49,0.5,"POLYGON((76.125 28.125,76.125 28.375,76.375 28...",Haryana,7,2021,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
9998754,1834585365,2021-07-01,2021-07-31 00:00:00,n2o,0.000507,0.1,0.49,0.5,"POLYGON((76.125 28.125,76.125 28.375,76.375 28...",Haryana,7,2021,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
9998758,1834585365,2021-07-01,2021-07-31 00:00:00,co2e_100yr,9.667294,0.1,0.49,0.5,"POLYGON((76.125 28.125,76.125 28.375,76.375 28...",Haryana,7,2021,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."


In [7]:
def add_state_to_geojson(df):
    updated_geojson_list = []

    for _, row in df.iterrows():
        # Parse the GeoJSON string to a dictionary
        geojson_data = json.loads(row['geojson'])
        state_name = row['state']

        # Update properties
        geojson_data['properties']['state'] = state_name
        updated_geojson_list.append(geojson_data)

    return updated_geojson_list

In [8]:
india_df['geojson'] = add_state_to_geojson(india_df)

# Display or save updated GeoJSON data as needed


/tmp/ipykernel_16874/2334827511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_df['geojson'] = add_state_to_geojson(india_df)


In [43]:
def create_emissions_map_plotly(data_df):
    """
    Creates an interactive choropleth map of emissions data for Indian states with side labels
    and prominent state borders
    
    Parameters:
    data_df (pd.DataFrame): DataFrame containing emissions data with columns:
        - state: State name
        - gas: Type of gas
        - emissions_quantity: Emission amount
        - geojson: GeoJSON string or dictionary of state boundaries
    
    Returns:
    dict: Dictionary of Plotly figures keyed by gas type
    """
    import numpy as np
    from shapely.geometry import shape, Point
    import json
    from collections import defaultdict
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # Process the data into state-wise emissions
    emissions_by_state = defaultdict(dict)
    # geojson_data = {"type": "FeatureCollection", "features": []}
    processed_states = set()
    
    def get_feature_boundary_point(feature, offset_direction='right'):
        """
        Calculate a point beside the state boundary for label placement
        """
        try:
            geometry = shape(feature['geometry'])
            bounds = geometry.bounds
            
            # Get the rightmost or leftmost point of the boundary
            if offset_direction == 'right':
                x = bounds[2]  # maxx
                y = (bounds[1] + bounds[3]) / 2  # middle y
            else:
                x = bounds[0]  # minx
                y = (bounds[1] + bounds[3]) / 2  # middle y
                
            # Add a small offset
            offset = 0.3  # Adjust this value to control label distance from boundary
            if offset_direction == 'right':
                x += offset
            else:
                x -= offset
                
            return [x, y]
        except:
            return None
    
    # Process data and calculate label positions
    state_label_positions = {}
    
    for _, row in data_df.iterrows():
        state = row['state']
        gas = row['gas']
        emissions = float(row['emissions_quantity'])
        
        if gas not in emissions_by_state[state] or emissions > emissions_by_state[state][gas]:
            emissions_by_state[state][gas] = emissions
        
        if state not in processed_states:
            try:
                if isinstance(row['geojson'], str):
                    state_geojson = json.loads(row['geojson'])
                else:
                    state_geojson = row['geojson']
                
                # Calculate label position for right side of state
                label_pos = get_feature_boundary_point(state_geojson, 'right')
                if label_pos:
                    state_label_positions[state] = label_pos
                
                state_geojson['properties'] = {
                    'state': state,
                    'name': state
                }
                geojson_data['features'].append(state_geojson)
                processed_states.add(state)
                
            except (json.JSONDecodeError, KeyError) as e:
                print(f"Error processing GeoJSON for state {state}: {e}")
                continue

    gas_types = ['co2', 'ch4', 'n2o', 'co2e_100yr', 'co2e_20yr']
    
    def create_gas_map(gas_type):
        # Create DataFrame for this gas type
        data = []
        for state, emissions in emissions_by_state.items():
            if gas_type in emissions:
                data.append({
                    'state': state,
                    'emissions': emissions[gas_type]
                })
        
        if not data:
            print(f"No data found for {gas_type}")
            return None
            
        df = pd.DataFrame(data)
        
        # Create figure with secondary axis for labels
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # Add choropleth map with enhanced borders
        choropleth = go.Choropleth(
            geojson=geojson_data,
            locations=df['state'],
            z=df['emissions'],
            colorscale='Reds',
            featureidkey='properties.state',
            name=f'{gas_type.upper()} Emissions',
            colorbar=dict(
                title=f'{gas_type.upper()} Emissions',
                thickness=20,
                len=0.75,
                y=0.5
            ),
            marker=dict(
                line=dict(
                    width=2,  # Increased border width
                    color='rgb(50, 50, 50)'  # Darker border color
                )
            ),
            hovertemplate="<b>%{location}</b><br>" +
                         f"{gas_type.upper()} Emissions: %{{z:,.2f}}<br>" +
                         "<extra></extra>"
        )
        
        # Add state labels using scattergeo
        lat_list = []
        lon_list = []
        text_list = []
        
        for state in df['state']:
            if state in state_label_positions:
                lon, lat = state_label_positions[state]
                lon_list.append(lon)
                lat_list.append(lat)
                text_list.append(state)
        
        # Add connecting lines between states and labels
        lines = go.Scattergeo(
            lon=[],
            lat=[],
            mode='lines',
            line=dict(
                width=0.5,
                color='gray'
            ),
            showlegend=False
        )
        
        # Add labels
        labels = go.Scattergeo(
            lon=lon_list,
            lat=lat_list,
            text=text_list,
            mode='text',
            textfont=dict(
                size=10,
                color='black',
                family='Arial'
            ),
            textposition='middle right',
            hoverinfo='skip',
            showlegend=False
        )
        
        # Add all traces to figure
        fig.add_trace(choropleth)
        fig.add_trace(lines)
        fig.add_trace(labels)
        
        # Update layout with enhanced geo settings
        fig.update_layout(
            title=dict(
                text=f'Maximum {gas_type.upper()} Emissions by State in India',
                x=0.5,
                xanchor='center',
                font=dict(size=20)
            ),
            paper_bgcolor='white',
            plot_bgcolor='white',
            margin=dict(t=50, l=0, r=0, b=0),
            height=800,
            geo=dict(
                visible=True,
                showcoastlines=True,
                coastlinecolor="rgb(50, 50, 50)",  # Darker coastline color
                coastlinewidth=2,  # Increased coastline width
                showland=True,
                landcolor="lightgray",
                showocean=True,
                oceancolor="lightblue",
                showlakes=True,
                lakecolor="lightblue",
                showcountries=True,
                countrycolor="rgb(50, 50, 50)",  # Darker country border color
                countrywidth=2,  # Increased country border width
                showsubunits=True,
                subunitcolor="rgb(50, 50, 50)",  # Darker state border color
                subunitwidth=2,  # Increased state border width
                center=dict(lat=20.5937, lon=78.9629),
                projection_scale=4,
                fitbounds="locations",
                resolution=50
            )
        )
        
        return fig

    # Create maps for all gas types
    maps = {}
    for gas in gas_types:
        map_fig = create_gas_map(gas)
        if map_fig:
            maps[gas] = map_fig
    
    return maps

In [44]:
def process_emissions_data(raw_data):
    """
    Process raw emissions data into the required format
    
    Parameters:
    raw_data (pd.DataFrame): Raw emissions data
    
    Returns:
    pd.DataFrame: Processed data with required columns
    """
    # Create a copy of the data with required columns
    required_columns = ['state', 'gas', 'emissions_quantity', 'geojson']
    processed_data = raw_data[required_columns].copy()
    
    # Ensure emissions_quantity is numeric
    processed_data['emissions_quantity'] = pd.to_numeric(processed_data['emissions_quantity'], errors='coerce')
    
    return processed_data

In [45]:
# Make sure to install shapely if you haven't already
# pip install shapely

processed_data = process_emissions_data(india_df)
maps = create_emissions_map_plotly(processed_data)
maps['n2o'].show()

In [39]:
maps['ch4'].show()

In [40]:
maps['co2'].show()

In [41]:
maps['co2e_100yr'].show()

In [42]:
maps['co2e_20yr'].show()